In [1]:
import os
import torch
import argparse
import numpy as np
import pandas as pd
from motif_util import *

from util import * 
from model import *

from pandas import DataFrame
from transformers import AutoConfig, BertConfig

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

/home/lijiahao/anaconda3/envs/tensorflow_pytorch_python/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
parser = argparse.ArgumentParser()
parser.add_argument('--mer',              type=int,  default=3,  help='')
parser.add_argument('--epoches',              type=int,  default=30,  help='')
parser.add_argument('--batch_size',           type=int,  default=16,  help='')
parser.add_argument('--max_length',           type=int,  default=200, help='')
parser.add_argument('--learning_rate',        type=float, default=1e-4, help="")
parser.add_argument('--model_path',           type=str,  default="../DNA-BERT/3-new-12w-0", help='')
parser.add_argument('--ind_filename',  type=str,  default="../dataset/enhancer_3-mer_DNABERT_ind.txt", help='')
parser.add_argument('--tra_filename',  type=str,  default="../dataset/enhancer_3-mer_DNABERT_tra.txt", help='')

args = parser.parse_args(args=[]) 

In [3]:
# L2-normalization average
mers = [3]
pre_label_list, real_label_list = [], []
for mer in mers:
    args.mer = mer
    args.model_path = "../DNA-BERT/{}-new-12w-0".format(mer)
    args.ind_filename = "../dataset/enhancer_{}-mer_DNABERT_ind.txt".format(mer)
    args.tra_filename = "../dataset/enhancer_{}-mer_DNABERT_tra.txt".format(mer)
    
    # construct config for Bert in order to output the attention of 12 layers
    # construct the model of BERT
    config = BertConfig.from_pretrained(args.model_path, output_attentions=True)
    model = C_Bert_2FC_average_atten.from_pretrained(args.model_path, config=config).to(device)
    
    # load model after fine-tuning
    model_dict = torch.load("fine-trained_model/C_Bert_2FC_average_{}-mer.pt".format(mer))
    model.load_state_dict(model_dict)

    # loader training dataset
    tra_dataloader = getData(args, split=False, validation=False, shuffle=False)
    # tra_atten_scores = output_logit_label_atten_head(model, tra_dataloader, args) 
    tra_atten_scores = output_logit_label_atten(model, tra_dataloader, args) 

    np.save("./atten/C_Bert_2FC_average_atten_{}-mer_tra".format(mer), tra_atten_scores)
    

    print("len: {}".format(tra_atten_scores.shape))

Some weights of the model checkpoint at ../DNA-BERT/3-new-12w-0 were not used when initializing C_Bert_2FC_average_atten: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing C_Bert_2FC_average_atten from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing C_Bert_2FC_average_atten from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of C_Bert_2FC_average_atten were not initialized from the model checkpoint at ../DNA-BERT/3-new-12w-0 and are newly ini

len: (2976, 200)
